In [5]:
# Some standard imports
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx


In [6]:
# Super Resolution model definition in PyTorch
import torch.nn as nn
import torch.nn.init as init


class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

# Create the super-resolution model by using the above model definition.
torch_model = SuperResolutionNet(upscale_factor=3)

In [7]:
# Load pretrained model weights
model_url = 'https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
batch_size = 1    # just a random number

# Initialize model with the pretrained weights
map_location = lambda storage, loc: storage
if torch.cuda.is_available():
    map_location = None
torch_model.load_state_dict(model_zoo.load_url(model_url, map_location=map_location))

# set the model to inference mode
torch_model.eval()

Downloading: "https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth" to /home/awf/.cache/torch/checkpoints/superres_epoch100-44c6958e.pth


SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pixel_shuffle): PixelShuffle(upscale_factor=3)
)

In [17]:
# Input to the model
x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)
torch_out = torch_model(x)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution_noparams.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=False,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                'output' : {0 : 'batch_size'}})
import random
random.random()

0.18146938375342359

In [34]:
import onnx
model = onnx.load("../../test/onnx2k/super_resolution.onnx")
onnx.checker.check_model(model)
print(onnx.helper.printable_graph(model.graph))


graph torch-jit-export (
  %input[FLOAT, batch_sizex1x224x224]
) initializers (
  %conv1.bias[FLOAT, 64]
  %conv1.weight[FLOAT, 64x1x5x5]
  %conv2.bias[FLOAT, 64]
  %conv2.weight[FLOAT, 64x64x3x3]
  %conv3.bias[FLOAT, 32]
  %conv3.weight[FLOAT, 32x64x3x3]
  %conv4.bias[FLOAT, 9]
  %conv4.weight[FLOAT, 9x32x3x3]
) {
  %9 = Conv[dilations = [1, 1], group = 1, kernel_shape = [5, 5], pads = [2, 2, 2, 2], strides = [1, 1]](%input, %conv1.weight, %conv1.bias)
  %10 = Relu(%9)
  %11 = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [1, 1, 1, 1], strides = [1, 1]](%10, %conv2.weight, %conv2.bias)
  %12 = Relu(%11)
  %13 = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [1, 1, 1, 1], strides = [1, 1]](%12, %conv3.weight, %conv3.bias)
  %14 = Relu(%13)
  %15 = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [1, 1, 1, 1], strides = [1, 1]](%14, %conv4.weight, %conv4.bias)
  %16 = Constant[value = <Tensor>]()
  %17 = Reshape(%15, %16)
  %18 = T

In [37]:
!pip install --upgrade git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-m0skcp_r
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-m0skcp_r
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-3.4.0-py3-none-any.whl size=1296698 sha256=1f39329d66047b20ed5bf5eba6316b17e046b64fe985ec07e2a15f4e1c23bbc8
  Stored in directory: /tmp/pip-ephem-wheel-cache-psqkrxyg/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 3.4.0
    Uninstalling transformers-3.4.0:
      Successfully uninstalled transformers-3.4.0


In [65]:
from transformers.convert_graph_to_onnx import load_graph_from_args,convert_pytorch
from pathlib import Path

nlp = load_graph_from_args("feature-extraction", "pt", "bert-base-cased")
!mkdir /tmp/bert-base-cased
convert_pytorch(nlp, opset=11, output=Path("/tmp/bert-base-cased/file.onnx"), use_external_format=True)
!ls -l /tmp/bert-base-cased

Loading pipeline (model: bert-base-cased, tokenizer: bert-base-cased)
Using framework PyTorch: 1.5.1
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Found output output_1 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']
total 423160
-rw-rw-r-- 1 awf awf  2359296 Nov  6 17:40 1661
-rw-rw-r-- 1 awf awf  2359296 Nov  6 17:40 1662
-rw-rw-r-- 1 awf awf  2359296 Nov  6 17:40 1663
-rw-rw-r-- 1 awf awf  2359296 Nov  6 17:40 1671
-rw-rw-r-- 1 awf awf  9437184 Nov  6 17:40 1673
-rw-rw-r-- 1 awf awf  9437184 Nov  6 17:40 1674
-rw-rw-r-- 1 awf awf  2359296 Nov  6 17:40 1676
-rw-rw-r-- 1 awf awf  2359296 Nov  6 17:40 1677
-rw-rw-r-- 1 awf awf  2359296

In [63]:
!echo 'FILES:' `ls ../../obj/bert-base-cased.onnx`
model = onnx.load("../../obj/bert-base-cased.onnx/file.onnx")
#onnx.checker.check_model(model)
s=onnx.helper.printable_graph(model.graph)
with open('../../obj/bert-base-cased.onnx/file.txt', 'w') as f:
    f.write(s)
print(s)

1030 = Constant[value = <Scalar Tensor []>]()
  %1031 = Add(%1029, %1030)
  %1032 = Sqrt(%1031)
  %1033 = Div(%1025, %1032)
  %1034 = Mul(%1033, %encoder.layer.6.attention.output.LayerNorm.weight)
  %1035 = Add(%1034, %encoder.layer.6.attention.output.LayerNorm.bias)
  %1037 = MatMul(%1035, %1763)
  %1038 = Add(%1037, %encoder.layer.6.intermediate.dense.bias)
  %1039 = Constant[value = <Scalar Tensor []>]()
  %1040 = Div(%1038, %1039)
  %1041 = Erf(%1040)
  %1042 = Constant[value = <Scalar Tensor []>]()
  %1043 = Add(%1041, %1042)
  %1044 = Mul(%1038, %1043)
  %1045 = Constant[value = <Scalar Tensor []>]()
  %1046 = Mul(%1044, %1045)
  %1048 = MatMul(%1046, %1764)
  %1049 = Add(%1048, %encoder.layer.6.output.dense.bias)
  %1050 = Add(%1049, %1035)
  %1052 = ReduceMean[axes = [-1]](%1050)
  %1053 = Sub(%1050, %1052)
  %1054 = Cast[to = 1](%1053)
  %1056 = Pow(%1054, %1765)
  %1057 = ReduceMean[axes = [-1]](%1056)
  %1058 = Constant[value = <Scalar Tensor []>]()
  %1059 = Add(%1057, %105